# Web Scraping Practice

In [49]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pyshorteners 

##### Define GetSoup function for future use

In [3]:
def getSoup(url):
  try:
    resp=requests.get(url)
    resp.encoding='utf-8'
    if resp.status_code == 200:
      return BeautifulSoup(resp.text,'lxml')
    else:
      return 'Error:Status_code'+ str(resp.status_code)

  except Exception as e:
    return e

### E.SUN BANK Currency Exchange Rate ( With Url Shortener Module Practice)

In [ ]:
url='https://www.esunbank.com/zh-tw/personal/deposit/rate/forex/foreign-exchange-rates'
ResPage = getSoup(url)
ResPage

In [133]:
Table_ExchangeRate = ResPage.find_all('table', class_="l-exchangeRate__block")
len(Table_ExchangeRate)

1

##### Build up Column List


> Get Title List

In [134]:
TempList = Table_ExchangeRate[0].find('thead').find_all('th')
TitleList = [ item.text for item in TempList ];TitleList


['', '即期匯率', '網銀/App優惠', '現金匯率']

> Get subtitle list and content

In [135]:
BodyList = Table_ExchangeRate[0].find('tbody', class_='l-exchangeRate__table result').find_all('tr');len(BodyList)

31

> Build up complete column list (The first row of this Body Table is subtitle)

In [90]:
List_td = BodyList[0].find_all('td')
ColumnList = ['Country']

for strTitle in TitleList:
    if strTitle != '':
        ColumnList.append(f"{strTitle}({List_td[1].text})")
        ColumnList.append(f"{strTitle}({List_td[2].text})")

ColumnList.append('Historial Trend Chart')

ColumnList


['Country',
 '即期匯率(銀行買入)',
 '即期匯率(銀行賣出)',
 '網銀/App優惠(銀行買入)',
 '網銀/App優惠(銀行賣出)',
 '現金匯率(銀行買入)',
 '現金匯率(銀行賣出)',
 'Historial Trend Chart']

##### Get Row Content

In [ ]:
Content_td = BodyList[1].find_all('td')
Content_td

> Get complete country name and the link to "Historical trend chart" (use Pyshorteners to wrap up long Url)

In [110]:
Country_Name = ''
Url_TrendChart = ''
Type_tiny = pyshorteners.Shortener()

#Get Country Block
Column_Country = Content_td[0].find('div', class_='row').find_all('div')

#Short the url to Historical Trend Chart
strUrl = f"https://www.esunbank.com/{Column_Country[3].find('a')['href']}"
Url_TrendChart = Type_tiny.tinyurl.short(strUrl)

#Get country name (Chinese+English)
Country_Name = f"{Column_Country[1].text.replace('\r\n','').strip()}({Column_Country[2].text.replace('\r\n','').strip()})"


In [66]:
Url_TrendChart

'http://tinyurl.com/ylafkecl'

In [67]:
Country_Name

'美元(USD)'

> Get Exchange Rate Data

> Board Rate

In [83]:
Content_td[4]

<td class="l-exchangeRate__table--inner" colspan="2">
<div class="BBoardRate">31.45</div>
<div class="SBoardRate">31.55</div>
</td>

> Buy/Sell Increase/Decrease Rate

In [84]:
Content_td[5]

<td class="l-exchangeRate__table--inner text-highlight" colspan="2">
<div class="BuyIncreaseRate">31.485</div>
<div class="SellDecreaseRate">31.515</div>
</td>

> Cash Board Rate

In [85]:
Content_td[6]

<td class="l-exchangeRate__table--inner" colspan="2">
<div class="CashBBoardRate">31.2</div>
<div class="CashSBoardRate">31.75</div>
</td>

> List of Exchange Rate

In [87]:
print(Content_td[4].find('div', class_='BBoardRate').text)
print(Content_td[4].find('div', class_='SBoardRate').text)
print(Content_td[5].find('div', class_='BuyIncreaseRate').text)
print(Content_td[5].find('div', class_='SellDecreaseRate').text)
print(Content_td[6].find('div', class_='CashBBoardRate').text)
print(Content_td[6].find('div', class_='CashSBoardRate').text)


31.45
31.55
31.485
31.515
31.2
31.75


> Get Complete Row Data with Country Name, Exchange Rate and the link to Historial Trend Chart

In [89]:
RowData = []
RowData.append(Country_Name)
RowData.append(Content_td[4].find('div', class_='BBoardRate').text)
RowData.append(Content_td[4].find('div', class_='SBoardRate').text)
RowData.append(Content_td[5].find('div', class_='BuyIncreaseRate').text)
RowData.append(Content_td[5].find('div', class_='SellDecreaseRate').text)
RowData.append(Content_td[6].find('div', class_='CashBBoardRate').text)
RowData.append(Content_td[6].find('div', class_='CashSBoardRate').text)
RowData.append(Url_TrendChart)

RowData

['美元(USD)',
 '31.45',
 '31.55',
 '31.485',
 '31.515',
 '31.2',
 '31.75',
 'http://tinyurl.com/ylafkecl']

##### Traversal the entire table (Row 1,3,5,7~31)

In [141]:
Type_tiny = pyshorteners.Shortener()
Country_Name = ''
Url_TrendChart = ''

RowData = []
DataSet = []


for Row_CountryData in BodyList:

    #Skip the first row because we've already finish column list
    if(BodyList.index(Row_CountryData) == 0):continue
    elif(BodyList.index(Row_CountryData) % 2 == 1):

        #Reset variable
        Country_Name = ''
        Url_TrendChart = ''
        RowData = []

        #Get Row Data
        Content_td = Row_CountryData.find_all('td')
    
        #Get Country Block
        Column_Country = Content_td[0].find('div', class_='row').find_all('div')
        
        #Short the url to Historical Trend Chart
        strUrl = f"https://www.esunbank.com/{Column_Country[3].find('a')['href']}"
        Url_TrendChart = Type_tiny.tinyurl.short(strUrl)

        #Get country name (Chinese+English)
        Country_Name = f"{Column_Country[1].text.replace('\r\n','').strip()}({Column_Country[2].text.replace('\r\n','').strip()})"

        #Build up the entire row data in RowData
        RowData.append(Country_Name)
        RowData.append(Content_td[4].find('div', class_='BBoardRate').text)
        RowData.append(Content_td[4].find('div', class_='SBoardRate').text)
        RowData.append(Content_td[5].find('div', class_='BuyIncreaseRate').text)
        RowData.append(Content_td[5].find('div', class_='SellDecreaseRate').text)
        RowData.append(Content_td[6].find('div', class_='CashBBoardRate').text)
        RowData.append(Content_td[6].find('div', class_='CashSBoardRate').text)
        RowData.append(Url_TrendChart)

        DataSet.append(RowData)

DataSet
        


[['美元(USD)',
  '31.45',
  '31.55',
  '31.485',
  '31.515',
  '31.2',
  '31.75',
  'http://tinyurl.com/ylafkecl'],
 ['人民幣(CNY)',
  '4.341',
  '4.391',
  '4.356',
  '4.376',
  '4.267',
  '4.427',
  'http://tinyurl.com/ywlgnnjb'],
 ['港幣(HKD)',
  '3.995',
  '4.055',
  '4.017',
  '4.033',
  '3.925',
  '4.085',
  'http://tinyurl.com/ynwwr3ee'],
 ['日圓(JPY)',
  '0.212',
  '0.216',
  '0.2132',
  '0.2148',
  '0.211',
  '0.218',
  'http://tinyurl.com/yl9r84pj'],
 ['歐元(EUR)',
  '34.06',
  '34.46',
  '34.18',
  '34.34',
  '33.76',
  '34.76',
  'http://tinyurl.com/ypy9j5f7'],
 ['澳幣(AUD)',
  '20.65',
  '20.85',
  '20.7',
  '20.8',
  '20.35',
  '21.15',
  'http://tinyurl.com/yoh4n4er'],
 ['加拿大幣(CAD)',
  '23.27',
  '23.45',
  '23.315',
  '23.405',
  '22.96',
  '23.76',
  'http://tinyurl.com/ym7mrd6g'],
 ['英鎊(GBP)',
  '39.6',
  '40',
  '39.72',
  '39.88',
  '39.3',
  '40.3',
  'http://tinyurl.com/ysffcpb7'],
 ['南非幣(ZAR)',
  '1.609',
  '1.709',
  '1.634',
  '1.684',
  '',
  '',
  'http://tinyurl.com/yl4m

##### Export Data as Excel file

In [142]:
df = pd.DataFrame(DataSet,columns=ColumnList)
df

,Country,即期匯率(銀行買入),即期匯率(銀行賣出),網銀/App優惠(銀行買入),網銀/App優惠(銀行賣出),現金匯率(銀行買入),現金匯率(銀行賣出),Historial Trend Chart
0,美元(USD),31.45,31.55,31.485,31.515,31.2,31.75,http://tinyurl.com/ylafkecl
1,人民幣(CNY),4.341,4.391,4.356,4.376,4.267,4.427,http://tinyurl.com/ywlgnnjb
2,港幣(HKD),3.995,4.055,4.017,4.033,3.925,4.085,http://tinyurl.com/ynwwr3ee
3,日圓(JPY),0.212,0.216,0.2132,0.2148,0.211,0.218,http://tinyurl.com/yl9r84pj
4,歐元(EUR),34.06,34.46,34.18,34.34,33.76,34.76,http://tinyurl.com/ypy9j5f7
5,澳幣(AUD),20.65,20.85,20.7,20.8,20.35,21.15,http://tinyurl.com/yoh4n4er
6,加拿大幣(CAD),23.27,23.45,23.315,23.405,22.96,23.76,http://tinyurl.com/ym7mrd6g
7,英鎊(GBP),39.6,40,39.72,39.88,39.3,40.3,http://tinyurl.com/ysffcpb7
8,南非幣(ZAR),1.609,1.709,1.634,1.684,,,http://tinyurl.com/yl4mfusp
9,紐西蘭幣(NZD),19.24,19.48,19.3,19.42,,,http://tinyurl.com/yw574cnu


In [143]:
df.to_csv('./Output/Quiz_3/ESunBank_CurrencyExchangeRate.csv',encoding='utf-8-sig')